In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch
import timeit
from utils import get_prompt
from generation import vistral7b

model = vistral7b.model
tokenizer = vistral7b.tokenizer
device = vistral7b.device
device

c:\Users\nguyen\miniconda3\envs\ai2\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\nguyen\miniconda3\envs\ai2\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\nguyen\miniconda3\envs\ai2\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\nguyen\.cache\huggingface\token
Login successful


c:\Users\nguyen\miniconda3\envs\ai2\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You shouldn't move a model that is dispatched using accelerate hooks.


ValueError: `.to` is not supported for `4-bit` or `8-bit` bitsandbytes models. Please use the model as it is, since the model has already been set to the correct devices and casted to the correct `dtype`.

In [3]:
system_prompt = """
Bạn là LegalBizAI, AI tư vấn Luật Doanh Nghiệp Việt Nam từ Team 3. Hãy trả lời câu hỏi về Luật Doanh Nghiệp Việt Nam.
Hướng dẫn trả lời:
•	Về bản thân và người sáng tạo: Trả lời như nội dung phần SYSTEM, không cần cú pháp Trích dẫn luật, Trả lời.
•	Câu hỏi không liên quan: Từ chối trả lời, nêu lý do LegalBizAI chỉ hỗ trợ Luật Doanh Nghiệp.
•	Câu hỏi liên quan: Trả lời dựa trên Căn cứ luật dưới đây với các điều khoản điểm, các điều được phân cách với nhau bằng chuỗi ---. Câu trả lời gồm chia làm 2 đầu mục "Trích dẫn luật" và "Trả lời". Phần trích dẫn luật in ra đầy đủ nội toàn bộ dung luật cần thiết để trả lời câu hỏi. Xuống dòng phân cách rõ ràng. Giải thích chi tiết lý do từ căn cứ luật, in đậm chi tiết quan trọng."""

custom_prompt_template = """Câu hỏi: {question}
Căn cứ luật:
{law_content}
"""


def generate_response(input_text: str, max_length: int = 2000) -> str:
    conversation = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": input_text
        },
    ]
    input_ids = tokenizer.apply_chat_template(
        conversation, return_tensors="pt",
        add_generation_prompt=True).to(device)
    print("Input token count=", input_ids.size(1))
    out_ids = model.generate(
        input_ids=input_ids,
        max_new_tokens=8000,
    )
    print(input_text)
    start_idx = input_ids.size(1)
    text = tokenizer.batch_decode(out_ids[:, start_idx:],
                                  skip_special_tokens=True)[0].strip()
    return text


prompt = get_prompt(
    "Hội đồng giải thể doanh nghiệp do Nhà nước nắm giữ 100% vốn điều lệ có được quyền sử dụng con dấu của doanh nghiệp hay không?",
    prompt_template=custom_prompt_template,
)
elapsed_time = timeit.timeit(
    lambda: print("VISTRAL ANSER:", generate_response(prompt)),
    number=1,
)
print(f"Elapsed time: {elapsed_time} seconds")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Input token count= 1104
Câu hỏi: Hội đồng giải thể doanh nghiệp do Nhà nước nắm giữ 100% vốn điều lệ có được quyền sử dụng con dấu của doanh nghiệp hay không?
Căn cứ luật:
Điều 43. Hội đồng giải thể doanh nghiệp do Nhà nước nắm giữ 100% vốn điều lệ
Khoản 1. Người có thẩm quyền quyết định giải thể doanh nghiệp thành lập Hội đồng giải thể doanh nghiệp. Hội đồng giải thể có chức năng tham mưu cho người quyết định giải thể về việc tổ chức thực hiện giải thể doanh nghiệp. Thành phần Hội đồng giải thể thực hiện theo quy định tại khoản 2, khoản 3 Điều này.
Khoản 2. Hội đồng giải thể của doanh nghiệp do Thủ tướng Chính phủ quyết định thành lập gồm đại diện các cơ quan sau:
Điểm a) Người đứng đầu cơ quan được giao thực hiện chức năng đại diện chủ sở hữu là Chủ tịch Hội đồng giải thể doanh nghiệp;
Điểm b) Đại diện các Bộ: Kế hoạch và Đầu tư, Tài chính, Lao động - Thương binh và Xã hội;
Điểm c) Đại diện các tổ chức đại diện cho người lao động tại doanh nghiệp bị giải thể;
Điểm d) Chủ tịch Hội đồn

In [3]:
get_prompt("Vô ý làm chết người tấn công mình",
           prompt_template=custom_prompt_template)

'Câu hỏi: Vô ý làm chết người tấn công mình\nCăn cứ luật:\nĐiều 173. Trách nhiệm của Kiểm soát viên\nKhoản 1. Tuân thủ đúng pháp luật, Điều lệ công ty, nghị quyết Đại hội đồng cổ đông và đạo đức nghề nghiệp trong thực hiện quyền và nghĩa vụ được giao.\nKhoản 2. Thực hiện quyền và nghĩa vụ được giao một cách trung thực, cẩn trọng, tốt nhất nhằm bảo đảm lợi ích hợp pháp tối đa của công ty.\nKhoản 3. Trung thành với lợi ích của công ty và cổ đông; không lạm dụng địa vị, chức vụ và sử dụng thông tin, bí quyết, cơ hội kinh doanh, tài sản khác của công ty để tư lợi hoặc phục vụ lợi ích của tổ chức, cá nhân khác.\nKhoản 4. Nghĩa vụ khác theo quy định của Luật này và Điều lệ công ty.\nKhoản 5. Trường hợp vi phạm quy định tại các khoản 1, 2, 3 và 4 Điều này mà gây thiệt hại cho công ty hoặc người khác thì Kiểm soát viên phải chịu trách nhiệm cá nhân hoặc liên đới bồi thường thiệt hại đó. Thu nhập và lợi ích khác mà Kiểm soát viên có được do vi phạm phải hoàn trả cho công ty.\nKhoản 6. Trường hợ

In [11]:
print(
    get_prompt(
        "Phòng Đăng ký kinh doanh phải chuyển tình trạng pháp lý của doanh nghiệp tư nhân trong Cơ sở dữ liệu quốc gia về đăng ký doanh nghiệp sang tình trạng đang làm thủ tục giải thể khi nào?",
        prompt_template=custom_prompt_template,
    )
)

Câu hỏi: Phòng Đăng ký kinh doanh phải chuyển tình trạng pháp lý của doanh nghiệp tư nhân trong Cơ sở dữ liệu quốc gia về đăng ký doanh nghiệp sang tình trạng đang làm thủ tục giải thể khi nào?
Căn cứ luật:
Điều 41. Tình trạng pháp lý của doanh nghiệp
Khoản 1. “Tạm ngừng kinh doanh” là tình trạng pháp lý của doanh nghiệp đang trong thời gian thực hiện tạm ngừng kinh doanh theo quy định tại khoản 1 Điều 206 Luật Doanh nghiệp. Ngày chuyển tình trạng pháp lý “Tạm ngừng kinh doanh” là ngày doanh nghiệp đăng ký bắt đầu tạm ngừng kinh doanh. Ngày kết thúc tình trạng pháp lý “Tạm ngừng kinh doanh” là ngày kết thúc thời hạn tạm ngừng kinh doanh mà doanh nghiệp đã thông báo hoặc ngày doanh nghiệp đăng ký tiếp tục kinh doanh trước thời hạn đã thông báo.
Khoản 2. “Không còn hoạt động kinh doanh tại địa chỉ đã đăng ký” là tình trạng pháp lý của doanh nghiệp mà qua công tác kiểm tra, xác minh của Cơ quan quản lý thuế và các đơn vị có liên quan không tìm thấy doanh nghiệp tại địa chỉ đã đăng ký. Thô

In [1]:
print(
    get_prompt(
        "Hội đồng giải thể doanh nghiệp do Nhà nước nắm giữ 100% vốn điều lệ có được quyền sử dụng con dấu của doanh nghiệp hay không?",
        prompt_template=custom_prompt_template,
    )
)

NameError: name 'get_prompt' is not defined